# Java Threads and Concurrency Utilities

## Chapter - 3
## Waiting and Notification

Java provides a small API that supports communication between threads. Using this API, one thread waits for a condition (a prerequisite for continued execution) to exist. In the future, another thread will create the condition and then notify the waiting thread. In this chapter, I introduce you to this API.


#### Wait-and-Notify API Tour
The java.lang.Object class provides a Wait-and-Notify API that consists of three wait() methods, one notify() method, and one notifyAll() method. The wait() methods wait for a condition to exist; the notify() and notifyAll() methods notify waiting threads when the condition exists:
* __void wait():__ Cause the current thread to wait until another thread invokes the notify() or notifyAll() method for this object, or for some other thread to interrupt the current thread while waiting.
* __void wait(long timeout):__ Cause the current thread to wait until another thread invokes the notify() or notifyAll() method for this object, or for the specified amount of time measured in milliseconds (identified by timeout) to pass, or for some other thread to interrupt the current thread while waiting. This method throws java.lang.IllegalArgumentException when timeout is negative.
* __void wait(long timeout, int nanos):__ Cause the current thread to wait until another thread invokes the notify() or notifyAll() method for this object, or for the specified amount of time measured in milliseconds (identified by timeout) plus nanoseconds (identified by nanos) to pass, or for some other thread to interrupt the current thread while waiting. This method throws IllegalArgumentException when timeout is negative, nanos is negative, or nanos is greater than 999999.
* __void notify():__ Wake up a single thread that’s waiting on this object’s monitor. If any threads are waiting on this object, one of them is chosen to be awakened. The choice is arbitrary and occurs at the discretion of the implementation. The awakened thread will not be able to proceed until the current thread relinquishes the lock on this object. The awakened thread will compete in the usual manner with any other threads that might be actively competing to synchronize on this object; for example, the awakened thread enjoys no reliable privilege or disadvantage in being the next thread to lock this object.
* __void notifyAll():__ Wake up all threads that are waiting on this object’s monitor. The awakened threads will not be able to proceed until the current thread relinquishes the lock on this object. The awakened threads will compete in the usual manner with any other threads that might be actively competing to synchronize on this object; for example, the awakened threads enjoy no reliable privilege or disadvantage in being the next thread to lock this object.

__All five methods must be called from within a synchronized context (the current thread must be the owner of the object’s monitor); otherwise, java.lang.IllegalMonitorStateException is thrown.__

__Never call a wait() method outside of a loop.__ 

The loop tests the condition before and after the wait() call. testing the condition before calling wait() ensures liveness. if this test was not present, and if the condition held and notify() had been called prior to wait() being called, it’s unlikely that the waiting thread would ever wake up. retesting the condition after calling wait() ensures safety. if retesting didn’t occur, and if the condition didn’t hold after the thread had awakened from the wait() call (perhaps another thread called notify() accidentally when the condition didn’t hold), the thread would proceed to destroy the lock’s protected invariants.

#### Producers and Consumers
A classic example of thread communication involving conditions is the relationship between a producer thread and a consumer thread. The producer thread produces data items to be consumed by the consumer thread. Each produced data item is stored in a shared variable.

In [1]:
class Shared {
    private char c;
    private volatile boolean writeable = true;

    synchronized void setSharedChar(char c) {
        while (!writeable)
            try {
                wait(100);
            } catch (InterruptedException ie) {
            }
        this.c = c;
        writeable = false;
        notify();
    }

    synchronized char getSharedChar() {
        while (writeable)
            try {
                wait(100);
            } catch (InterruptedException ie) {
            }
        writeable = true;
        notify();
        return c;
    }
}

In [2]:
class Producer extends Thread {
    private final Shared s;

    Producer(Shared s) {
        this.s = s;
    }

    @Override
    public void run() {
        for (char ch = 'A'; ch <= 'Z'; ch++) {
            synchronized (s) {
                s.setSharedChar(ch);
                System.out.println(ch + " produced by producer.");
            }
        }
    }
}

In [3]:
class Consumer extends Thread {
    private final Shared s;

    Consumer(Shared s) {
        this.s = s;
    }

    @Override
    public void run() {
        char ch;
        do {
            synchronized (s) {
                ch = s.getSharedChar();
                System.out.println(ch + " consumed by consumer.");
            }
        } while (ch != 'Z');
    }
}

In [4]:
Shared s = new Shared();
new Producer(s).start();
new Consumer(s).start();

A produced by producer.
A consumed by consumer.
B produced by producer.
B consumed by consumer.
C produced by producer.
C consumed by consumer.
D produced by producer.
D consumed by consumer.
E produced by producer.
E consumed by consumer.
F produced by producer.
F consumed by consumer.
G produced by producer.
G consumed by consumer.
H produced by producer.
H consumed by consumer.
I produced by producer.
I consumed by consumer.
J produced by producer.
J consumed by consumer.
K produced by producer.
K consumed by consumer.
L produced by producer.
L consumed by consumer.
M produced by producer.
M consumed by consumer.
N produced by producer.
N consumed by consumer.
O produced by producer.
O consumed by consumer.
P produced by producer.
P consumed by consumer.
Q produced by producer.
Q consumed by consumer.
R produced by producer.
R consumed by consumer.
S produced by producer.
S consumed by consumer.
T produced by producer.
T consumed by consumer.
U produced by producer.
U consumed by co

In [5]:
public class AnotherPC {
    volatile int i = 1;
    volatile Character c = 'a';
    volatile boolean state = true;

    synchronized void printAlphabet() {
        try {
            while (!state) {
                wait();
            }
        } catch (InterruptedException e) {
        }
        System.out.println(Thread.currentThread().getName() + "\t:\t" + c);
        state = false;
        c++;
        notifyAll();
    }

    synchronized void printNumbers() {
        try {
            while (state) {
                wait();
            }
        } catch (InterruptedException e) {
        }
        System.out.println(Thread.currentThread().getName() + "\t:\t" + i);
        state = true;
        i++;
        notifyAll();
    }
}

In [7]:
AnotherPC threadClass = new AnotherPC();

Runnable runnable1 = () -> {
    int i = 0;
    while (i < 26) {
        threadClass.printAlphabet();
        i++;
    }

};

Runnable runnable2 = () -> {
    int i = 0;
    while (i < 26) {
        threadClass.printNumbers();
        i++;
    }

};

Thread t1 = new Thread(runnable1);
t1.setName("Thread_Alphabet");

Thread t2 = new Thread(runnable2);
t2.setName("Thread_Number");

t1.start();
t2.start();

Thread_Alphabet	:	a
Thread_Number	:	1
Thread_Alphabet	:	b
Thread_Number	:	2
Thread_Alphabet	:	c
Thread_Number	:	3
Thread_Alphabet	:	d
Thread_Number	:	4
Thread_Alphabet	:	e
Thread_Number	:	5
Thread_Alphabet	:	f
Thread_Number	:	6
Thread_Alphabet	:	g
Thread_Number	:	7
Thread_Alphabet	:	h
Thread_Number	:	8
Thread_Alphabet	:	i
Thread_Number	:	9
Thread_Alphabet	:	j
Thread_Number	:	10
Thread_Alphabet	:	k
Thread_Number	:	11
Thread_Alphabet	:	l
Thread_Number	:	12
Thread_Alphabet	:	m
Thread_Number	:	13
Thread_Alphabet	:	n
Thread_Number	:	14
Thread_Alphabet	:	o
Thread_Number	:	15
Thread_Alphabet	:	p
Thread_Number	:	16
Thread_Alphabet	:	q
Thread_Number	:	17
Thread_Alphabet	:	r
Thread_Number	:	18
Thread_Alphabet	:	s
Thread_Number	:	19
Thread_Alphabet	:	t
Thread_Number	:	20
Thread_Alphabet	:	u
Thread_Number	:	21
Thread_Alphabet	:	v
Thread_Number	:	22
Thread_Alphabet	:	w
Thread_Number	:	23
Thread_Alphabet	:	x
Thread_Number	:	24
Thread_Alphabet	:	y
Thread_Number	:	25
Thread_Alphabet	:	z
Thread_Number	

#### Summary
Java provides an API that supports communication between threads. This API consists of Object’s three wait() methods, one notify() method, and one notifyAll() method. The wait() methods wait for a condition to exist; notify() and notifyAll() notify waiting threads when the condition exists.


The wait(), notify(), and notifyAll() methods are called from within a synchronized block that synchronizes on the same object as the object on which they are called. Because of spurious wakeups, wait() is called from a while loop that reexecutes wait() while the condition doesn’t hold.


A classic example of thread communication involving conditions is the relationship between a producer thread and a consumer thread. The producer thread produces data items to be consumed by the consumer thread. Each produced data item is stored in a shared variable.


To overcome problems such as consuming a data item that hasn’t been produced, the producer thread must wait until it’s notified that the previously produced data item has been consumed, and the consumer thread must wait until it’s notified that a new data item has been produced.